In [8]:
import datetime
from hashlib import sha256

MAX_ENTRIES_AMOUNT = 5
def isCryptographicPuzzleSolved(aBlock):
    return aBlock.hash() < (2**256) / aBlock.header['difficulty'] - 1
    
class Block: 
    def __init__(self, entries):
        self.header = {
            'prev_hash': 0,
            'nonce': 0,
            'timestamp': None,
            'entries_amount': len(entries),
            'difficulty': 1
        }
        if (len(entries) <= MAX_ENTRIES_AMOUNT):
            self.entries = entries
        else:
            raise 'Exceeding max block size'
            
    def hash(self):
        return int(sha256(repr(self.header).encode('utf-8') + repr(self.entries).encode('utf-8')).hexdigest(), 16)
        
    def __str__(self):
        entries = ",".join(self.entries)
        return """
        'block_hash': {0}
        
        'header': {{
            'prev_hash':{1}
            'nonce': {2}
            'timestamp': {3}
            'entries_amount': {4}
            'difficulty': {5}
        }}
        
        'entries': [
            {6}
        ]
        """.format(hex(self.hash()), hex(self.header['prev_hash']), self.header['nonce'], self.header['timestamp'], self.header['entries_amount'], self.header['difficulty'], entries)
    
class Blockchain:
    def __init__(self):
        self.blocks = []
        self.last_block_hash = 0
        
    def addBlock(self, newBlock):
        if (self.isBlockValid(newBlock)):
            self.blocks.append(newBlock)
            self.last_block_hash = newBlock.hash()
            return True
        return False
    
    def isBlockValid(self, block):
        return block.header['prev_hash'] == self.last_block_hash and isCryptographicPuzzleSolved(block)
    
    def getLastHash(self):
        return self.last_block_hash
    
    def printBlockChain(self):
        for block in self.blocks:
            print ('-----------------------------------------------------------------------------')
            print (block)
            print ('-----------------------------------------------------------------------------')

In [9]:
def mine(prev_hash, block):
    block.header['prev_hash'] = prev_hash
    block.header['timestamp'] = datetime.datetime.now()
    while not isCryptographicPuzzleSolved(block):
        block.header['nonce'] += 1
        block.header['timestamp'] = datetime.datetime.now()

In [10]:
# Example creating a blockchain and adding one block only. 
# The second block doesn't get added because it does not match the prev_hash
coolBlockChain = Blockchain()
newBlock = Block(["{'user_id': 'user_1', 'user_data': 'data_1'}", "{'user_id': 'user_2', 'user_data': 'data_2'}"])

mine(coolBlockChain.getLastHash(), newBlock)

coolBlockChain.addBlock(newBlock)
coolBlockChain.addBlock(newBlock)

False

In [11]:
coolBlockChain.printBlockChain()

-----------------------------------------------------------------------------

        'block_hash': 0x73419558f6d81c4037b3f9d382515dca14a320c198a7afeae0899d7b6728e778
        
        'header': {
            'prev_hash':0x0
            'nonce': 0
            'timestamp': 2021-04-27 20:07:28.264181
            'entries_amount': 2
            'difficulty': 1
        }
        
        'entries': [
            {'user_id': 'user_1', 'user_data': 'data_1'},{'user_id': 'user_2', 'user_data': 'data_2'}
        ]
        
-----------------------------------------------------------------------------


In [12]:
def adjustDifficulty(prevDifficulty, elapsedTime, targetTime, blocksProcessed):
    return prevDifficulty * (blocksProcessed/elapsedTime)*targetTime

In [13]:
def generateBlock(blockchain, difficulty, i):
    newBlock = Block(["{{'user_id': 'user_{0}', 'user_data': 'data_{0}'}}".format(i)])
    newBlock.header['difficulty'] = difficulty
    mine(blockchain.getLastHash(), newBlock)
    blockchain.addBlock(newBlock)
    
    print ('{0} | Block {2} mined with difficulty: {1}!'.format(datetime.datetime.now(), difficulty, i))
    print (newBlock)

In [14]:
# Example of how difficulty is adjusted.
TARGET_TIME_IN_SECONDS = 60 # Every 60s a new block is mined in avg.
BLOCKS_PROCESSED_TO_ADJUST_DIFFICULTY = 256 # Every 256 blocks difficulty is adjusted to match TARGET_TIME.

elapsedTimeInSeconds = 0
blocksProccesed = 0

awesomeBlockchain = Blockchain()
difficulty = 1

startTime = datetime.datetime.now()

genesisBlock = Block(["{'user_id': 'user_genesis', 'user_data': 'data_genesis'}"])
mine(awesomeBlockchain.getLastHash(), genesisBlock)
awesomeBlockchain.addBlock(genesisBlock)

blocksProccesed += 1
i = 0
# Simulation of a working blockchain.
# A random block is generated in each iteration and difficulty is adjusted dynamically
while len(awesomeBlockchain.blocks) < 120:
    if blocksProccesed >= BLOCKS_PROCESSED_TO_ADJUST_DIFFICULTY:
        elapsedTimeInSeconds = (datetime.datetime.now() - startTime).total_seconds()
        difficulty = adjustDifficulty(difficulty, elapsedTimeInSeconds, TARGET_TIME_IN_SECONDS, blocksProccesed)
        blocksProccesed = 0
        startTime = datetime.datetime.now()
        
    generateBlock(awesomeBlockchain, difficulty, i)
    i += 1
    blocksProccesed += 1

2021-04-27 20:07:39.887504 | Block 0 mined with difficulty: 1!

        'block_hash': 0x556fc0057c685eaf3249b05156c463aacf4a55850f43cf22bd7cda5ca164f655
        
        'header': {
            'prev_hash':0xdaabc42f2934562f468bcdf7caead38df14bc861206b58bd448a492798d3fa98
            'nonce': 0
            'timestamp': 2021-04-27 20:07:39.887504
            'entries_amount': 1
            'difficulty': 1
        }
        
        'entries': [
            {'user_id': 'user_0', 'user_data': 'data_0'}
        ]
        
2021-04-27 20:07:39.887504 | Block 1 mined with difficulty: 1!

        'block_hash': 0x70333ad22061c759f918ec1c724c59405f170b2b18862152b49acc5da3b179d1
        
        'header': {
            'prev_hash':0x556fc0057c685eaf3249b05156c463aacf4a55850f43cf22bd7cda5ca164f655
            'nonce': 0
            'timestamp': 2021-04-27 20:07:39.887504
            'entries_amount': 1
            'difficulty': 1
        }
        
        'entries': [
            {'user_id': 'use

In [15]:
# Example on how multiple clients issue transactions to the system and those are batched together in blocks with max size 5.

def receiveTransactionFromClient(i):
    return "{{'user_id': 'user_{0}', 'user_data': 'data_{0}'}}".format(i)

amazingBlockchain = Blockchain()
difficulty = 1

genesisBlock = Block(['transaction_genesis'])
mine(amazingBlockchain.getLastHash(), genesisBlock)
amazingBlockchain.addBlock(genesisBlock)

# Receive 5 transactions and batch them together.
transactions = []
for i in range(5):
    transactions.append(receiveTransactionFromClient(i))

firstBlock = Block(transactions)
mine(amazingBlockchain.getLastHash(), firstBlock)
amazingBlockchain.addBlock(firstBlock)

transactions = []
# Receive 3 transactions and batch them together.
transactions = []
for i in range(3):
    transactions.append(receiveTransactionFromClient(i))

secondBlock = Block(transactions)
mine(amazingBlockchain.getLastHash(), secondBlock)
amazingBlockchain.addBlock(secondBlock)

amazingBlockchain.printBlockChain()

-----------------------------------------------------------------------------

        'block_hash': 0xf2c68d89fe9588e170814d1ac5155795370364f35da93917000141eaf9baa656
        
        'header': {
            'prev_hash':0x0
            'nonce': 0
            'timestamp': 2021-04-27 20:07:48.101896
            'entries_amount': 1
            'difficulty': 1
        }
        
        'entries': [
            transaction_genesis
        ]
        
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------

        'block_hash': 0x660ee41d96ee4aa8f8765cbd7b84e33fe25591d6dbab4dc4d77b1017a23b99fe
        
        'header': {
            'prev_hash':0xf2c68d89fe9588e170814d1ac5155795370364f35da93917000141eaf9baa656
            'nonce': 0
            'timestamp': 2021-04-27 20:07:48.101896
            'entries_amount': 5
            'difficulty': 1
        }
        
        'entries': [
        

In [16]:
# Example dispatching a block to be mine by multiple workers using multithreading module.
import threading
import copy

runningOutOfNamesBlockchain = Blockchain()

def mineConcurrent(prev_hash, block, blockchain, i):
        block.header['difficulty'] = 100000
        block.header['nonce'] = i**7
        block.header['prev_hash'] = prev_hash
        block.header['timestamp'] = datetime.datetime.now()
        while not isCryptographicPuzzleSolved(block):
                block.header['nonce'] += 1
                block.header['timestamp'] = datetime.datetime.now()
        if (blockchain.addBlock(block)):
            print ('Block mined by thread {0}'.format(i))

def dispatchBlock(W, block):
    threads = []
    block_mined = None
    last_hash = runningOutOfNamesBlockchain.getLastHash()
    for i in range(W):
        t = threading.Thread( target=mineConcurrent, args=(last_hash, copy.deepcopy(firstBlock), runningOutOfNamesBlockchain, i) )
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

# Receive 5 transactions and batch them together.
transactions = []
for i in range(5):
    transactions.append(receiveTransactionFromClient(i))

firstBlock = Block(transactions)

# Dispatch the block to 3 workers to mine the work.
dispatchBlock(3, firstBlock)

runningOutOfNamesBlockchain.printBlockChain()


Block mined by thread 2
-----------------------------------------------------------------------------

        'block_hash': 0xb69b5a5c3a958619c786e759f346663c709f891c57423c297479d128305
        
        'header': {
            'prev_hash':0x0
            'nonce': 2846
            'timestamp': 2021-04-27 20:07:49.287404
            'entries_amount': 5
            'difficulty': 100000
        }
        
        'entries': [
            {'user_id': 'user_0', 'user_data': 'data_0'},{'user_id': 'user_1', 'user_data': 'data_1'},{'user_id': 'user_2', 'user_data': 'data_2'},{'user_id': 'user_3', 'user_data': 'data_3'},{'user_id': 'user_4', 'user_data': 'data_4'}
        ]
        
-----------------------------------------------------------------------------
